In [1]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
langchain_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4
from langchain_community.document_loaders import WebBaseLoader

def get_retriever_from_blog_posts():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = Chroma(embedding_function=embeddings)

    urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]

    for url in urls:
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=("post-content", "post-title", "post-header")
                )
            ),
        )

        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        splits = text_splitter.split_documents(docs)
        vectorstore.add_documents(splits)

        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 6}
        )

    return retriever


retriever = get_retriever_from_blog_posts()

docs_retrived = retriever.invoke("agent memory")
docs_retrived


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(id='27eb0525-bbb2-4a5f-9310-c4758995a26a', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The design of generative agents combines LLM with memory, planning and reflection mechanisms to enable agents to behave conditioned on past experience, as well as to interact with other agents.'),
 Document(id='d4a0c8f8-84c4-4533-bf04-b4b2cb67a165', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.'),
 Document(id='e9fa6ab0-2e98-4641-a632-29d92e6a950c', metadata={'source': 'https://lilianweng.githu

In [3]:
# 5-2 참고 코드
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class RelevanceResult(BaseModel):
    relevance: bool = Field(description="query relevance to the text chunk")


def is_relevant(query, docs):
    parser = JsonOutputParser(pydantic_object=RelevanceResult)

    prompt = PromptTemplate(
        template="""
    You are an expert at evaluating information retrieval quality.
    Given a user query and a retrieved text chunk, determine if the chunk is relevant to the query.

    # Format: {format_instructions}
    # User Query: {query}
    # Retrieved Chunk: {text}
        """,
        partial_variables={"format_instructions": parser.get_format_instructions()},
        input_variables=["query", "text"],
    )

    chain = prompt | llm | parser
    return chain.invoke({"query": query, "text": docs})

# for all fail
for doc in docs_retrived:
    print(is_relevant("카카오", doc.page_content))

for doc in docs_retrived:
    print("\n\n##########",is_relevant("agent memory", doc.page_content), doc.page_content)


def get_only_relevance_docs(query, retriever):
    docs = retriever.invoke(query)
    relevant_docs = []
    for doc in docs:
        result = is_relevant(query, doc.page_content)
        if result['relevance']:
            relevant_docs.append(doc)
    return "\n\n".join(doc.page_content for doc in relevant_docs)



{'relevance': False}
{'relevance': False}
{'relevance': False}
{'relevance': False}
{'relevance': False}
{'relevance': False}


########## {'relevance': True} The design of generative agents combines LLM with memory, planning and reflection mechanisms to enable agents to behave conditioned on past experience, as well as to interact with other agents.


########## {'relevance': True} Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.

Each element is an observation, an event directly provided by the agent.
- Inter-agent communication can trigger new natural language statements.


Retrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.


########## {'relevance': True} Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve a

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    template="""
You are an expert assistant. Answer the following User Query using only the information provided in the Retrieved Chunk. If the Retrieved Chunk does not contain enough information to answer the query, say "The information is not available in the provided text."

Retrieved Chunk: {context}
User Query: {question}

Your Answer:
    """,
    input_variables=["context", "question"],
)

rag_chain = (
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

query = "What is Task Decomposition?"
relevance_docs = get_only_relevance_docs(query, retriever)

for chunk in rag_chain.stream({"context": relevance_docs, "question": query}):
    print(chunk, end="", flush=True)

Task decomposition is the process by which an agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. It can be achieved through various methods, including simple prompting by a language model, task-specific instructions, or human inputs. Additionally, it involves planning ahead and allows the agent to reflect on and refine past actions to improve the quality of future results.